### Analysing the optimal neighbourhood to open a new restaurant in the Manhattan Borough of New York using Foursquare and New York Location Data 

#### Project Key 
1.0 - Introduction 
1.1 - Data 
1.2 - Methodology 
1.3 - Results 
1.4 - Discussion 
1.5 - Conclusion 

#### 1.0 Introduction 

For a large scale restaurant chain or business the location of a new restaurent can be very important, where a restuarent is located may determine how many people can access and visit it and therefore, the revenue a restaurant can generate. The number of competing restaurants in an area may also be a determining factor of how succesful a restaurant will be. My project intends to provide important information to stakeholders who may be looking to open a new restaurant in the Manhattan Borough of New York City. 

Using foursquare location data and New York City postcode data I will investigating the ideal neighbourhood to open a new restaurant in Manhattan. To assess this problem I will be using segmentation and clustering techniques to Identify help which neighbourhood would be a good location for a new restaurant. I will be assessing the frequency of restauarants in each neighbourhood in a given radius and I aim to find a neighbourhood which has a lower frequency of restuarent venus and therefore, could be an excellent location for a new restaurent. 

#### 1.1 Data

As outlined in the introduction section of my project, the data that will be used for my project will be New York location data, including information for boroughs, neighbourhoods and Lattitude/Longitude. Initially the data the New York data must be preprocessed to be usuable in my project, this process is undertaken in the below sections of code. In addition to this, I will require the use of Foursquare API to link my New York data set to obtain venue information. The New York data set is freely accesible and has been converted to a JSON file. The Foursquare data is obtained by the creation of a Foursquare developer account which will allow me to make a several calls to the API to obtain venue information for the Manhattan Borough of New York.   

### 1.2 Methodology  

To conduct an investigation into which neighbourhoods in Manhattan would be favourable for a restaurant to open I had to process and analyse the new york neighbourhood and borough data that I had. Initially, I had to import all relevant packages to allow me to analyse the data, and following this, data had to be transformed to a dataframe that could be used and analysed. I then had to extract the neighbourhoods that were part of the Manhattan borough as these would be used for my analysis (Figure 1.1).

The next step was to link Foursquare to my New York data set to allow me to obtain information on the venues in New York and more specifically, Manhattan. This was achieved through linking my Foursquare credentials and would enable me to make calls to the API to obtain information about venues in neighbourhoods in Manhattan. Through several steps (shown in the code below), I was able to segment the data to obtain the the types and frequency of venues with a 500 metre radius of each neighbourhood (Figure 1.2). Once the frequency of each type of venue had been determined this was presented in a dataframe which showed the mean frequency of each type of venue in the set radius. This then enabled me to determine the top 10 most common venues for each neighbourhood by the frequency that they had occured (Figure 1.3). I then looked to determine which neighbourhoods did not contain a restaurant in the top 10 most common venues and could therefore be good locations for a new restaurant to open, a new dataframe was created which contained the neighbourhoods that did not have a restaurant present(Figure 1.4).

### 1.3 Results 

The results of the analysis of New York data showed that 40 neighbourhoods were present in Manhattan, and linking these neighbourhoods (listed in figure 1.2) to venue data using the Foursquare API produced a dataframe containing the top 10 most common venues for each of these listed neighbourhoods. 37 Neighbourhoods in total contained a restaurant in the top 10 most common venues within a 500 metre radius, leaving 3 neighbourhoods which did not contain a restaurant within the top 10 most common venues. The neighbourhoods containing no restaurant were: Battery Park City, Marble Hill and Stuyvesan Town (Figure 1.5). 

### 1.4 Discussion 

The results of this project showed three potential locations for a new restaurant in the Manhattan area of New York City, based on a restaurant not being a top 10 most common venue for these neighbourhoods. As outlined in the results section, these neighbourhoods were: Battery Park City, Marble City and Stuyvesan Town. I believe these to be favourable locations for a future restaurant to be opened based on the analysis conducted. It is likely that a restauarant built in one of these neighbourhoods is likely to face less competition from other restaurant venues and potentially higher popularity due to the lack of this particular service in these neighbourhoods compared with others. However, further analysis could be conducted using a wider radius to provide a more detailed picture of these neighbourhoods. In addition to this, further analysis could be conducted to determine what type of restaurant is most popular in Manhattan and is therefore likely to thrive in a new neighbourhood. The investigation 

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!pip install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!pip install folium 
import folium 

print('Libraries imported.')


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes
     |████████████████████████████████| 94 kB 5.2 MB/s  eta 0:00:01
Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data) 
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [4]:
neighborhoods_data = newyork_data['features'] 
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [5]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)  
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)  

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [9]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Figure 1.0 - Neighborhoods in New York 

In [11]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [12]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Figure 1.1 - Map of Neighbourhoods in Manhattan 

In [13]:
CLIENT_ID = 'MWQXGMTZFC1WBGSEMO0IITKLO4EICIXRLLYL5B2TDQQOLLFU' 
CLIENT_SECRET = 'WQQTDACOUZEDZ1KWZPMELODEMIDR3IJM5K2STOEYD3QINVZY' 
VERSION = '20180605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MWQXGMTZFC1WBGSEMO0IITKLO4EICIXRLLYL5B2TDQQOLLFU
CLIENT_SECRET:WQQTDACOUZEDZ1KWZPMELODEMIDR3IJM5K2STOEYD3QINVZY


In [14]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [15]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=MWQXGMTZFC1WBGSEMO0IITKLO4EICIXRLLYL5B2TDQQOLLFU&client_secret=WQQTDACOUZEDZ1KWZPMELODEMIDR3IJM5K2STOEYD3QINVZY&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '602b8258e7297327caae58be'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list]) 
    nearby_venues.columns= ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [24]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [25]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,73,73,73,73,73,73
Carnegie Hill,89,89,89,89,89,89
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,42,42,42,42,42,42
East Village,100,100,100,100,100,100


In [26]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe 

In [27]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe 

#### Figure 1.2  - Dataframe showing the frequency of venues in each Neighbourhood in Manhattan 

In [29]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0     Coffee Shop  0.08
1           Hotel  0.07
2  Clothing Store  0.05
3            Park  0.05
4             Gym  0.05


----Carnegie Hill----
               venue  freq
0        Coffee Shop  0.09
1               Café  0.06
2        Yoga Studio  0.03
3          Bookstore  0.03
4  French Restaurant  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1   Chinese Restaurant  0.05
2  American Restaurant  0.05
3                  Bar  0.05
4   Seafood Restaurant  0.05


----Chelsea----
                 venue  freq
0          Coffee Shop  0.06
1               Bakery  0.05
2          Art Gallery  0.05
3  American Restaurant  0.04
4            Wine Shop  0.03


----Chinatown----
                venue  freq
0              Bakery  0.08
1  Chinese Restaurant  0.07
2        Dessert Shop  0.05
3   Hotpot Restaurant  0.04
4        Cocktail Bar  0.04


----Civic Center----
                  venue  freq
0           

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Hotel,Park,Gym,Clothing Store,Memorial Site,Playground,Sandwich Place,Gourmet Shop,Plaza
1,Carnegie Hill,Coffee Shop,Café,Bookstore,French Restaurant,Cosmetics Shop,Yoga Studio,Gym,Wine Shop,Pizza Place,Japanese Restaurant
2,Central Harlem,African Restaurant,French Restaurant,Seafood Restaurant,Bar,Chinese Restaurant,Art Gallery,Cosmetics Shop,American Restaurant,Fried Chicken Joint,Caribbean Restaurant
3,Chelsea,Coffee Shop,Bakery,Art Gallery,American Restaurant,French Restaurant,Seafood Restaurant,Ice Cream Shop,Wine Shop,Cupcake Shop,Cosmetics Shop
4,Chinatown,Bakery,Chinese Restaurant,Dessert Shop,Cocktail Bar,Hotpot Restaurant,Spa,American Restaurant,Bubble Tea Shop,Salon / Barbershop,Ice Cream Shop


In [87]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Hotel,Park,Gym,Clothing Store,Memorial Site,Playground,Sandwich Place,Gourmet Shop,Plaza
1,Carnegie Hill,Coffee Shop,Café,Bookstore,French Restaurant,Cosmetics Shop,Yoga Studio,Gym,Wine Shop,Pizza Place,Japanese Restaurant
2,Central Harlem,African Restaurant,French Restaurant,Seafood Restaurant,Bar,Chinese Restaurant,Art Gallery,Cosmetics Shop,American Restaurant,Fried Chicken Joint,Caribbean Restaurant
3,Chelsea,Coffee Shop,Bakery,Art Gallery,American Restaurant,French Restaurant,Seafood Restaurant,Ice Cream Shop,Wine Shop,Cupcake Shop,Cosmetics Shop
4,Chinatown,Bakery,Chinese Restaurant,Dessert Shop,Cocktail Bar,Hotpot Restaurant,Spa,American Restaurant,Bubble Tea Shop,Salon / Barbershop,Ice Cream Shop
5,Civic Center,Coffee Shop,Gym / Fitness Center,Spa,Cocktail Bar,Hotel,Yoga Studio,French Restaurant,Italian Restaurant,Bakery,Park
6,Clinton,Italian Restaurant,Gym / Fitness Center,Theater,American Restaurant,Sandwich Place,Coffee Shop,Wine Shop,Hotel,Cocktail Bar,Spa
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Sandwich Place,Park,Deli / Bodega,Latin American Restaurant,Steakhouse,Cocktail Bar,Taco Place
8,East Village,Bar,Pizza Place,Mexican Restaurant,Italian Restaurant,Wine Bar,Korean Restaurant,Coffee Shop,Ice Cream Shop,Seafood Restaurant,Salon / Barbershop
9,Financial District,Coffee Shop,Italian Restaurant,Cocktail Bar,Bar,Pizza Place,Gym / Fitness Center,Gym,Salad Place,American Restaurant,Mexican Restaurant


#### Figure 1.3  - Dataframe outlining the top 10 most common venues for Neighbourhoods in Manhattan

In [92]:
neighborhoods_amended = neighborhoods_venues_sorted[~neighborhoods_venues_sorted["1st Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant"])]  
neighborhoods_amended1 = neighborhoods_amended[~neighborhoods_amended["2nd Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant","American Restaurant","Seafood Restaurant"])]    
neighborhoods_amended2 = neighborhoods_amended1[~neighborhoods_amended1["3rd Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant","American Restaurant","Seafood Restaurant"])]   
neighborhoods_amended3 = neighborhoods_amended2[~neighborhoods_amended2["4th Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant","American Restaurant","Seafood Restaurant","French Restaurant"])]
neighborhoods_amended4 = neighborhoods_amended3[~neighborhoods_amended3["5th Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant","American Restaurant","Seafood Restaurant","French Restaurant"])]  
neighborhoods_amended5 = neighborhoods_amended4[~neighborhoods_amended4["6th Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant","American Restaurant","Seafood Restaurant","French Restaurant","Japanese Restaurant","Greek Restaurant"])] 
neighborhoods_amended6 = neighborhoods_amended5[~neighborhoods_amended5["7th Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant","American Restaurant","Seafood Restaurant","French Restaurant","Japanese Restaurant","Greek Restaurant","Latin American Restaurant"])] 
neighborhoods_amended7 = neighborhoods_amended6[~neighborhoods_amended6["8th Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant","American Restaurant","Seafood Restaurant","French Restaurant","Japanese Restaurant","Greek Restaurant","Latin American Restaurant"])]   
neighborhoods_amended8 = neighborhoods_amended7[~neighborhoods_amended7["9th Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant","American Restaurant","Seafood Restaurant","French Restaurant","Japanese Restaurant","Greek Restaurant","Latin American Restaurant"])]  
neighborhoods_final = neighborhoods_amended8[~neighborhoods_amended8["10th Most Common Venue"].isin(["African Restaurant","Italian Restaurant","Mexican Restaurant","Chinese Restaurant","Korean Restaurant","American Restaurant","Seafood Restaurant","French Restaurant","Japanese Restaurant","Greek Restaurant","Latin American Restaurant"])]   
neighborhoods_final

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Hotel,Park,Gym,Clothing Store,Memorial Site,Playground,Sandwich Place,Gourmet Shop,Plaza
22,Marble Hill,Sandwich Place,Coffee Shop,Gym,Discount Store,Yoga Studio,Diner,Supplement Shop,Donut Shop,Steakhouse,Kids Store
30,Stuyvesant Town,Park,Bar,Coffee Shop,Harbor / Marina,Gas Station,Boat or Ferry,Farmers Market,Gym / Fitness Center,Cocktail Bar,Heliport


#### Figure 1.4  -  Unassigned Dataframe showing all Neighbourhoods in Manhattan that do not contain a Restaurant in the top 10 most common venues 

In [93]:
neighborhoods_final.reset_index(drop=True, inplace=True)  
neighborhoods_final

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Hotel,Park,Gym,Clothing Store,Memorial Site,Playground,Sandwich Place,Gourmet Shop,Plaza
1,Marble Hill,Sandwich Place,Coffee Shop,Gym,Discount Store,Yoga Studio,Diner,Supplement Shop,Donut Shop,Steakhouse,Kids Store
2,Stuyvesant Town,Park,Bar,Coffee Shop,Harbor / Marina,Gas Station,Boat or Ferry,Farmers Market,Gym / Fitness Center,Cocktail Bar,Heliport


#### Figure 1.5 - Reassigned Dataframe outlining Neighbourhoods in Manhattan which do not contain a restaurant in the top 10 most common venues 

### 1.2 Methodology   

To conduct an investigation into which neighbourhoods in Manhattan would be favourable for a restaurant to open I had to process and analyse the new york neighbourhood and borough data that I had. Initially, I had to import all relevant packages to allow me to analyse the data, and following this, data had to be transformed to a dataframe that could be used and analysed. I then had to extract the neighbourhoods that were part of the Manhattan borough as these would be used for my analysis. 

The next step was to link Foursquare to my New York data set to allow me to obtain information on the venues in New York and more specifically, Manhattan. This was achieved through linking my Foursquare credentials and would enable me to make calls to the API to obtain information about venues in neighbourhoods in Manhattan. Through several steps (shown in the code above), I was able to segment the data to obtain the the types and frequency of venues with a 500 metre radius of each neighbourhood. 